In [1]:
import numpy as np
from ksp_interface import GuidanceHelperFunctions as ghf
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import krpc
import json
import time
import random

In [2]:
conn = krpc.connect(name='KRPC Test')
print(conn.krpc.get_status().version)
vessel = conn.space_center.active_vessel
kerbin = conn.space_center.bodies['Kerbin']
celest_refframe = kerbin.non_rotating_reference_frame # This doesn't seem to work any better really
vessel_refframe = vessel.reference_frame
session = (conn, vessel, celest_refframe)

0.5.4


In [13]:
def ksp2pyoptflight(coords: np.ndarray):
    coords = np.atleast_2d(coords)
    x = coords[:, 0]
    y = coords[:, 2]
    z = coords[:, 1]
    return np.column_stack((x, y, z))

In [22]:
nsamples = 10000
A_ref = (1.25/2)**2*np.pi

# All in KSP coordinate system!
start_time = time.time()
a = vessel.flight().speed_of_sound
rho = vessel.flight().atmosphere_density
pos = np.array(vessel.position(celest_refframe))
wind = np.cross(np.array(kerbin.angular_velocity(celest_refframe)), pos)

ebx = np.array(conn.space_center.transform_direction([1, 0, 0], vessel_refframe, celest_refframe))
eby = np.array(conn.space_center.transform_direction([0, 1, 0], vessel_refframe, celest_refframe))
ebz = np.array(conn.space_center.transform_direction([0, 0, 1], vessel_refframe, celest_refframe))

vel_dir_samples = np.empty((nsamples, 3))
mach_samples = np.empty((nsamples))
force_samples = np.empty((nsamples, 3))

for i in range(nsamples):
    M = random.uniform(0, 25)
    phi = random.uniform(0, np.pi/2)
    theta = random.uniform(-np.pi, np.pi)

    vel_dir = ebx*np.cos(theta)*np.sin(phi) + eby*np.cos(phi) + ebz*np.sin(theta)*np.sin(phi)
    vel_rel = vel_dir*M*a
    aero_force = vessel.flight(celest_refframe).simulate_aerodynamic_force_at(kerbin, pos, vel_rel+wind)

    mach_samples[i] = M
    vel_dir_samples[i] = vel_dir
    force_samples[i] = np.array(aero_force)

    if i%10 == 0:
        print(f'{100*(i+1)/nsamples:.1f} %, {time.time()-start_time:.2f} sec')

# use ebx, eby, ebz to generate velocity samples and aerodynamic force samples all in KSP coords
# convert vel dir samples, force samples, ebx to PyOptFlight frame
# Now calculate the current vehicle frame from eBx, eBy, eBz in PyOptFlight coords
# Do the force projections onto eBx, eBy, eBz and compute the cos angles using converted vel_dir_samples
# Need to back out C_A, C_Ny, C_Nz


0.0 %, 0.02 sec
0.1 %, 0.04 sec
0.2 %, 0.05 sec
0.3 %, 0.10 sec
0.4 %, 0.13 sec
0.5 %, 0.16 sec
0.6 %, 0.16 sec
0.7 %, 0.19 sec
0.8 %, 0.21 sec
0.9 %, 0.24 sec
1.0 %, 0.27 sec
1.1 %, 0.29 sec
1.2 %, 0.31 sec
1.3 %, 0.32 sec
1.4 %, 0.34 sec
1.5 %, 0.36 sec
1.6 %, 0.38 sec
1.7 %, 0.40 sec
1.8 %, 0.40 sec
1.9 %, 0.42 sec
2.0 %, 0.44 sec
2.1 %, 0.46 sec
2.2 %, 0.48 sec
2.3 %, 0.50 sec
2.4 %, 0.51 sec
2.5 %, 0.53 sec
2.6 %, 0.55 sec
2.7 %, 0.59 sec
2.8 %, 0.60 sec
2.9 %, 0.64 sec
3.0 %, 0.66 sec
3.1 %, 0.68 sec
3.2 %, 0.70 sec
3.3 %, 0.72 sec
3.4 %, 0.74 sec
3.5 %, 0.77 sec
3.6 %, 0.79 sec
3.7 %, 0.81 sec
3.8 %, 0.84 sec
3.9 %, 0.85 sec
4.0 %, 0.89 sec
4.1 %, 0.91 sec
4.2 %, 0.93 sec
4.3 %, 0.95 sec
4.4 %, 0.99 sec
4.5 %, 1.00 sec
4.6 %, 1.03 sec
4.7 %, 1.05 sec
4.8 %, 1.08 sec
4.9 %, 1.10 sec
5.0 %, 1.14 sec
5.1 %, 1.16 sec
5.2 %, 1.19 sec
5.3 %, 1.22 sec
5.4 %, 1.25 sec
5.5 %, 1.28 sec
5.6 %, 1.31 sec
5.7 %, 1.34 sec
5.8 %, 1.37 sec
5.9 %, 1.41 sec
6.0 %, 1.44 sec
6.1 %, 1.47 sec
6.2 %, 1

In [23]:
eBx = ksp2pyoptflight(eby)[0]
eBy = np.cross([0, 0, 1], eBx)
eBz = np.cross(eBx, eBy)
vel_dir_samples = ksp2pyoptflight(vel_dir_samples)
force_samples = ksp2pyoptflight(force_samples)
vrel_mag = mach_samples*a
FBx = force_samples @ eBx # dot product (vector projection)
FBy = force_samples @ eBy # dot product (vector projection)
FBz = force_samples @ eBz # dot product (vector projection)
C_A = FBx/(0.5*rho*vrel_mag**2*A_ref)
C_Ny = FBy/(0.5*rho*vrel_mag**2*A_ref)
C_Nz = FBz/(0.5*rho*vrel_mag**2*A_ref)
cos_AoA_x = vel_dir_samples @ eBx # dot product (cos law)
cos_AoA_y = vel_dir_samples @ eBy # dot product (cos law)
cos_AoA_z = vel_dir_samples @ eBz # dot product (cos law)

In [27]:

# —————————————————————————————————————
# 1) C_A vs cos_AoA_x & Mach
# —————————————————————————————————————
fig1 = go.Figure(data=go.Scatter3d(
    x=np.rad2deg(np.arccos(cos_AoA_x)),
    y=mach_samples,
    z=C_A,
    mode='markers',
    marker=dict(size=2)
))
fig1.update_layout(
    title="C_A vs cos(AoA)_x & Mach",
    scene=dict(
        xaxis_title="cos(AoA)_x",
        yaxis=dict(
            title="Mach",
            type="log"
        ),
        zaxis_title="C_A"
    )
)
fig1.show()


# —————————————————————————————————————
# 2) C_Ny vs cos_AoA_y & Mach
# —————————————————————————————————————
fig2 = go.Figure(data=go.Scatter3d(
    x=np.rad2deg(np.arccos(cos_AoA_y)),
    y=mach_samples,
    z=C_Ny,
    mode='markers',
    marker=dict(size=2)
))
fig2.update_layout(
    title="C_Ny vs cos(AoA)_y & Mach",
    scene=dict(
        xaxis_title="cos(AoA)_y",
        yaxis=dict(
            title="Mach",
            type="log"
        ),
        zaxis_title="C_Ny"
    )
)
fig2.show()


# —————————————————————————————————————
# 3) C_Nz vs cos_AoA_z & Mach
# —————————————————————————————————————
fig3 = go.Figure(data=go.Scatter3d(
    x=np.rad2deg(np.arccos(cos_AoA_z)),
    y=mach_samples,
    z=C_Nz,
    mode='markers',
    marker=dict(size=2)
))
fig3.update_layout(
    title="C_Nz vs cos(AoA)_z & Mach",
    scene=dict(
        xaxis_title="cos(AoA)_z",
        yaxis=dict(
            title="Mach",
            type="log"
        ),
        zaxis_title="C_Nz"
    )
)
fig3.show()

In [ ]:
ksp2pyoptflight(vel_dir_samples)

Need to collect the following data:
* Temperature vs altitude
* Density vs altitude (verify density equation)
* Stage masses
* Stage reference areas
* Stage thrust and Isp (vac and sl), would be nice to verift thrust and isp equations too
* Random velocity directions and magnitudes in hemisphere centered on vehicle y (defined by krpc)
* Sequence starting from vehicle y and rotating in plane to x and z, repeating for different velocities

In [ ]:
pos = np.array(vessel.position(celest_refframe))
pos/np.linalg.norm(pos)